# Installing necessary libraries for web scraping

In [1]:
#Installing necessary libraries for web scraping
!pip install selenium
!pip install BeautifulSoup4

# Importing necessary libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Webscraping data

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# makes a request to the web page and gets its HTML
r = requests.get(url)

# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.content)

# assigning to data frame

In [4]:
a = []
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
for link in soup.find_all('td'):
    a.append(link.get_text())
    if len(a) == 3:
        df_length = len(df)
        df.loc[df_length] = a
        a = []
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
7,M8A\n,Not assigned\n,Not assigned\n
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"


# Cleaning data

In [5]:
df2 = df.replace('\n','', regex=True)
df2 = df2.drop(index = range(180,191))
df2 = df2.drop(index = (df2[(df2['Borough']=='Not assigned')].index))
df2 = df2.reset_index()
df2

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Checking if there are existing Not assigned value in column, Neighborhood

In [6]:
df2.loc[df2['Neighborhood']=='Not assigned']

,index,PostalCode,Borough,Neighborhood


In [7]:
df2 = df2.reindex()
df2.shape

(103, 4)